# Cylinder data type

Next, we discuss all small details to include a new atomic ditribution for `CoupledDipole.jl`

---

## Step 1: Create New Shape
First, we need to define a struct that is a subtype of a `Dimension` data type. A cylinder is a 3D object, therefore, we define it as a sub type of `ThreeD` type.  
In practice, add the following line into `structs.jl` file:

`struct Cylinder <: ThreeD end`

Then, export it on the `CoupledDipole.jl` file:  
`export Cylinder`

## Step 2: Discover how to generate the new shape
Second step is to create the atoms distribution is space. We need to find properties that fit inside the `Atom` struct, that is, we need `r` (matrix of atom's positions), `N` (number of atoms), `sizes` (some numbers that represents the system size).

The easiest starting point is to prototype the data distribution with some plots, later, we figure out how to put on the right format.

My strategy is to create points inside a disc, then create another value for the heigh. 

The part of creating points in a disc needs some care, check [here](https://mathworld.wolfram.com/DiskPointPicking.html).

In [ ]:
using Plots

N = 10_000
R = 2
h = 10
r,θ = R*rand(N), 2π*rand(N)

x = sqrt.(r).*cos.(θ)
y = sqrt.(r).*sin.(θ)
z = -h.*rand(N) .+ h/2

fig1 = scatter(x,y, zcolor=r, aspect_ratio=:equal, label="", cbar=false)
fig2 = scatter(x,y,z, zcolor=r, aspect_ratio=:equal, label="", cbar=false)
plot(fig1, fig2, layout=(1,2), size=(600,400)) |> display

# Step 3: Define the Basic Constructor

Now that we know how to create the spatial distribution, we need to create a function the create individual atoms and other constructors related functions.

Create the file `src/atom_cylinder.jl`.  

Import these new file inside the `CoupledDipoles.jl` file, that is, add the command:  
`include("atom_cylinder.jl")`

Copy and Paste the file `atom_cube.jl` into `src/atom_cylinder.jl` changing the words `Cube` to `Cylinder`.

We change the basic definitions ad hoc: 

- change the inputs to get `R` and `h`
    ```
        function Atom(
        geometry::Cylinder,
        N::Int64,
        R::Union{Real,Integer},
        h::Union{Real,Integer};
        createFunction = ftn_AtomsOnCylinder::Function,
    )
    ```
- change the density
    ```
        ρ = N/( h*π*R^2 )
    ```

- pass `R` and `h` to create atoms
    ```
        r = get_atoms(dimensions, N, rₘᵢₙ; createFunction, R, h)
    ```
- create a function to create a *single* atom
    ```
    function ftn_AtomsOnCylinder(; kwargs...)
        R, h = kwargs[:R], kwargs[:h]
        
        r,θ = R*rand(), 2π*rand()

        x = sqrt(r)*cos(θ)
        y = sqrt(r)*sin(θ)
        z = -h*rand() + h/2

        return [x, y, z]
    end
    ```
- create a `sizes` tuple and create the `Atom` struct
```
    sizes = (R=Float64(R), h=Float64(h))
    return Atom(Cylinder(), r, N, sizes)
```

- create another constructor that has the atom's positions as input
    ```
    function Atom(geometry::Cylinder, r::Matrix, R::Union{Real,Integer}, h::Union{Real,Integer})
        N = size(r, 2) # remember to use each collum as a atom position
        return Atom(Cylinder(), r, N, (R=Float64(R), h=Float64(h)) )
    end
    ```

# Step 4: Make Pretty IO
Change the `src/IO.jl` file to have a pretty print message.  

```
function Base.show(io::IO, atoms::Atom{Cylinder})
    geometry_text = "Atoms on a $( highlight("Cylinder", :yellow) )"
    N_text = " with N=$(highlight(atoms.N, :yellow)) and "
    size_text = "[R=$(highlight(make_short(atoms.sizes[:R]), :yellow)), h=$(highlight(make_short(atoms.sizes[:h]), :yellow))]"
    return printstyled(io, geometry_text * N_text * size_text)
end
```

# Step 5: Test

Now we have the most basic functions working and we can create atoms natively.

In [ ]:
using CoupledDipoles
N = 5000
R = 2
h = 15
cylindric_cloud = Atom(Cylinder(), N, R, h)

In [ ]:
using Plots
x = cylindric_cloud.r[1, :]
y = cylindric_cloud.r[2, :]
z = cylindric_cloud.r[3, :]
r = sqrt.( x.^2 + y.^2 )

fig1 = scatter(x,y, zcolor=r, aspect_ratio=:equal, label="", cbar=false)
fig2 = scatter(x,y,z, zcolor=r, aspect_ratio=:equal, label="", cbar=false)
plot(fig1, fig2, layout=(1,2), size=(600,400)) |> display

# Step 6: Create Auxiliary Functions

Since we can create atoms, now we have to deal with auxiliary functions to process them.

## 6.1 Define the Volume 
- In file `formulas.jl`
    ```
    Volume_of(atoms::Atom{Cylinder}) = atoms.sizes[:h] * π * atoms.sizes[:R]^2
    ```

## 6.2 Create `cylinder_inputs` 

This function helps when we want to define N and $\rho$ and don't want to compute the exact dimensions of the cylinder

- In file `atoms.jl`: 
    ```
    function cylinder_inputs(N::Integer, ρ::Real; kwargs...)
        R =  get(kwargs, :R, NaN)
        h =  get(kwargs, :h, NaN)

        if (R < 0) && (h < 0)
            @error "Invalid inputs, R and h must be positives"
            R = 0
            h = 0
        elseif isnan(R) && isnan(h) # R and h not provided
            # assume that R=h ⇢ (h*πR^2 = πR^3) and solve for R
            R = cbrt(N / π * ρ)
            h = R
        elseif isnan(R) && (h ≠ -1.0) # R not provided
            R = sqrt(N / (π * ρ * h))
        elseif (R ≠ -1.0) && isnan(h) # h not provided
            h = sqrt(N / (π * ρ * R^2))
        end
        return (N, R, h)
    end
    ```

- In file `CoupledDipoles.jl`:  
    ```
    export cylinder_inputs
    ```

## 6.3 Define the Path Length 

We imagine a beam of light crossing the cloud, we need to specify how big is this path. I assume light in z-direction, because it is where the laser is (currently) pointing. Therefore, the path length is equal to the cylider's height.

- In file `formulas.jl` (this code is valid only for homogeneous distribution)
    ```
    function geometric_integral(atoms::Atom{Cylinder})
        @info "using the cylinder height as light path"
        return atoms.sizes[:h]
    end
    ```

- In file `IO.jl`
    ```
    Base.size(x::Atom{Cylinder}) = x.sizes[:h]
    ```

# Step 7: Finished

Now you are free to change the word

In [2]:
using CoupledDipoles
N = 150
ρ = 0.2
h = 15

# R will be computed automatically
cylindric_cloud = Atom(Cylinder(), cylinder_inputs(N,ρ; h=h)...)

Atoms on a Cylinder with N=150 and [R=3.99, h=15.0]